<a href="https://colab.research.google.com/github/Aditya-pixel/MLDL-Assign2/blob/master/MLDL_ASSIGNF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.data import imread
import matplotlib.pyplot as plt
from pathlib import Path
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import os
import random
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras import optimizers
from keras.layers import Activation, Dense
from keras.layers import Input
import io
from io import StringIO
import string
import pandas as pd
from sklearn.model_selection import train_test_split
import os, cv2, re, random
from random import shuffle
from tqdm import tqdm

import torch.nn as nn #neural networks mool
import torchvision #used for visualizing things editing data etc
import torchvision.transforms as transforms #cropping fitting etc
from keras.layers import Dropout


Using TensorFlow backend.


In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9988/868324/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1577886011&Signature=MLzxpskBhRMhqAtOZW788b%2FfLHVVkS6rVm2SOK9noJnOC40MnGSaSxD2UpqGFSPDFDtH%2FkLygegnIuGXGyXBZOyosFsveDoQAxZUAp2Bx7jTP5S7WfXbqNiYssqcNMNuzRfv%2F9x6wUi7UYbmIDePbiOK61d4I8Oo3TvOsapY7w7WcgnC78msKaKP2aX%2FwsmwdqhnLF1muqYX%2BYRFHWAg4LaZScSPx02QlnWEABcMNwGgL8Tt0oC2FXq6VJfO6AqjzX9RjHcve3ivfnq7Qqlhaf30avXCSOf7%2BBeW4isoeU0AmBGV%2B8f%2Fbo%2FDxhqT9zw9bZ3oK5jWfZ5Ft9uWgWBlSg%3D%3D&response-content-disposition=attachment%3B+filename%3Dairbus-ship-detection.zip" -O "airbus-ship-detection.zip" -c

--2019-12-29 13:42:25--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/9988/868324/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1577886011&Signature=MLzxpskBhRMhqAtOZW788b%2FfLHVVkS6rVm2SOK9noJnOC40MnGSaSxD2UpqGFSPDFDtH%2FkLygegnIuGXGyXBZOyosFsveDoQAxZUAp2Bx7jTP5S7WfXbqNiYssqcNMNuzRfv%2F9x6wUi7UYbmIDePbiOK61d4I8Oo3TvOsapY7w7WcgnC78msKaKP2aX%2FwsmwdqhnLF1muqYX%2BYRFHWAg4LaZScSPx02QlnWEABcMNwGgL8Tt0oC2FXq6VJfO6AqjzX9RjHcve3ivfnq7Qqlhaf30avXCSOf7%2BBeW4isoeU0AmBGV%2B8f%2Fbo%2FDxhqT9zw9bZ3oK5jWfZ5Ft9uWgWBlSg%3D%3D&response-content-disposition=attachment%3B+filename%3Dairbus-ship-detection.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30690510746 (29G) [application/zip]
Saving to: ‘airbus-ship-detection.zip’

airbus-ship

In [0]:
import os
import gc
print(os.listdir("/content"))
import numpy as np 
import pandas as pd
import time
gc.collect()

['.config', 'airbus-ship-detection.zip', 'sample_data']


115

In [0]:
!unzip /content/airbus-ship-detection.zip

In [0]:
train = pd.read_csv('/content/train_ship_segmentations_v2.csv')

In [0]:
ships = train[~train.EncodedPixels.isna()].ImageId.unique()
noships = train[train.EncodedPixels.isna()].ImageId.unique()

plt.bar(['Ships', 'No Ships'], [len(ships), len(noships)]);
plt.ylabel('Number of Images');

In [0]:
train['exist_ship'] = train['EncodedPixels'].fillna(0)
train.loc[train['exist_ship'] != 0 , 'exist_ship'] = 1

In [0]:
from keras.utils import np_utils
import numpy as np
from glob import glob

Train_path = '/content/train_v2'
Test_path = '/content/test_v2'

In [0]:
train_gp = train.groupby(['ImageId']).sum().reset_index()
train_gp.loc[train_gp['exist_ship']>0,'exist_ship']=1

train_sample = train_gp.sample(5000)
test_sample = train_gp.sample(1000)

In [0]:
def load_dataset(path):
    files_array = []
    if str(path) == str(Train_path):
        data = np.array(train_sample['ImageId'])
        data_targets = np_utils.to_categorical(np.array(train_sample['exist_ship']), 133)

        for idx, element in  enumerate(data): 
            files_array.append(Train_path + element)

        data = np.array(files_array)
    else:
        data = np.array(test_sample['ImageId'])
        data_targets = np_utils.to_categorical(np.array(test_sample['exist_ship']), 133)

        for idx, element in  enumerate(data): 
            files_array.append(Train_path + element)

        data = np.array(files_array)
    
    return data, data_targets

In [0]:
train_files, train_targets = load_dataset(Train_path)
test_files, test_targets = load_dataset(Test_path)

In [0]:
from keras.preprocessing import image 
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 


In [0]:
from numpy import genfromtxt
train_data = genfromtxt('/content/train_ship_segmentations_v2.csv', delimiter=',')

In [0]:
inputs = Input((768, 768, 3))
model = Sequential()
model.add(Conv2D(input_shape=(768,768,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Activation('softmax'))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)


In [0]:
train_tensors = paths_to_tensor(train_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255


In [0]:
from keras.callbacks import Callback
START = 0.5
END = 0.95
STEP = 0.05
N_STEPS = int((END - START) / STEP) + 2
DEFAULT_THRESHOLDS = np.linspace(START, END, N_STEPS)
DEFAULT_BETA = 1
DEFAULT_LOGS = {}
FBETA_METRIC_NAME = "val_fbeta"

# Some unit test constants
input_dim = 2
num_hidden = 4
num_classes = 2
batch_size = 5
train_samples = 20
test_samples = 20
SEED = 42
TEST_BETA = 2
EPOCHS = 5




# Notice that this callback only works with Keras 2.0.0


class FBetaMetricCallback(Callback):

    def __init__(self, beta=DEFAULT_BETA, thresholds=DEFAULT_THRESHOLDS):
        self.beta = beta
        self.thresholds = thresholds
        # Will be initialized when the training starts
        self.val_fbeta = None

    def on_train_begin(self, logs=DEFAULT_LOGS):
        """ This is where the validation Fbeta
        validation scores will be saved during training: one value per
        epoch.
        """
        self.val_fbeta = []

    def _score_per_threshold(self, predictions, targets, threshold):
        """ Compute the Fbeta score per threshold.
        """
        
        thresholded_predictions = (predictions > threshold).astype(int)
        return fbeta_score(targets, thresholded_predictions, beta=self.beta, average='micro')

    def on_epoch_end(self, epoch, logs=DEFAULT_LOGS):
        val_predictions = self.model.predict(self.validation_data[0])
        val_targets = self.validation_data[1]
        _val_fbeta = np.mean([self._score_per_threshold(val_predictions,
                                                        val_targets, threshold)
                              for threshold in self.thresholds])
        self.val_fbeta.append(_val_fbeta)
        print("Current F{} metric is: {}".format(str(self.beta), str(_val_fbeta)))
        return

    def on_train_end(self, logs=DEFAULT_LOGS):
        """ Assign the validation Fbeta computed metric to the History object.
        """
        self.model.history.history[FBETA_METRIC_NAME] = self.val_fbeta

In [0]:
from keras.callbacks import ModelCheckpoint  

epochs = 20

fbeta_metric_callback = FBetaMetricCallback(beta=2)
history = model.fit(train_tensors, train_targets, 
          validation_data=(test_tensors, test_targets),
          epochs=epochs, batch_size=20, callbacks=[fbeta_metric_callback])